# Data Cleaning with Pandas

In [1]:
import pandas as pd

## Scenario

As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

In [2]:
ls -l

total 176
-rw-r--r--  1 nawjux  staff     56 Apr  5 09:07 README.md
drwxr-xr-x  6 nawjux  staff    192 Apr  5 09:21 data/
-rw-r--r--  1 nawjux  staff  82875 Apr  5 11:29 pandas-3-data-cleaning.ipynb


### First, get the data!

When working on a project involving data that can fit on our computer, we store it in a `data` directory.

```bash
cd <project_directory>  # example: cd ~/flatiron_ds/pandas-3
mkdir data
cd data
```

Note that `<project_directory>` in angle brackets is a _placeholder_. You should type the path to the actual location on your computer where you're working on this project. Do not literally type `<project_directory>` and _do not type the angle brackets_. You can see an example in the _comment_ to the right of the command above.

Now, we'll need to download the two data files that we need. We can do this at the command line:

```bash
wget https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip
wget https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip
```

*Note:* If you do not have the `wget` command yet, you can install it: `brew install wget`.

Note that `%20` in a URL translates into a space. Even though you will *never put spaces in filenames*, you may need to deal with spaces that _other_ people have used in filenames.

There are two ways to handle the spaces in these filenames when referencing them at the command line.

#### 1. You can _escape_ the spaces by putting a backslash (`\`, remember _backslash is next to backspace_) before each one:

`unzip Real\ Property\ Sales.zip`

This is what happens if you tab-complete the filename in the terminal. Tab completion is your friend!

#### 2. You can put the entire filename in quotes:

`unzip "Real Property Sales.zip"`

Try unzipping these files with the `unzip` command. The `unzip` command takes one argument, the name of the tile that you want to unzip.


You can use tab completion (press the `tab` key after the first three letters) to fill in the names, including spaces. This will 

In [3]:
sales_df = pd.read_csv('data/EXTR_RPSale.csv')

/Users/nawjux/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Seeing pink? Warnings are useful!

Note the warning above: `DtypeWarning: Columns (1, 2) have mixed types.` Because we start with an index of zero, the columns that we're being warned about are actually the _second_ and _third_ columns, `sales_df['Major']` and `sales_df['Minor']`.

Looking at data, you can see some have SalePrice of 0, because it transfered ownership and wasn't sold per se. 

In [4]:
sales_df.head().T

,0,1,2,3,4
ExciseTaxNbr,2687551,1235111,2704079,2584094,1056831
Major,138860,664885,423943,403700,951120
Minor,110,40,50,715,900
DocumentDate,08/21/2014,07/09/1991,10/11/2014,01/04/2013,04/20/1989
SalePrice,245000,0,0,0,85000
RecordingNbr,20140828001436,199203161090,20141205000558,20130110000910,198904260448
Volume,,071,,,117
Page,,001,,,053
PlatNbr,,664885,,,951120
PlatType,,C,,,P


### Data overload?

That's a lot of columns. We're only interested in identifying the date, sale price, and square footage of each specific property. What can we do?

In [5]:
sales_df = sales_df[['Major', 'Minor', 'DocumentDate', 'SalePrice']]

In [6]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014336 entries, 0 to 2014335
Data columns (total 4 columns):
Major           object
Minor           object
DocumentDate    object
SalePrice       int64
dtypes: int64(1), object(3)
memory usage: 61.5+ MB


In [7]:
bldg_df = pd.read_csv('data/Residential Building.zip')

/Users/nawjux/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Another warning! Which column has index 11?

In [8]:
bldg_df.columns[11]

'ZipCode'

`ZipCode` seems like a potentially useful column. We'll need it to determine which house sales took place in Seattle.

In [48]:
bldg_df.head().T

,0,1,2,3,4
Major,4300,4610,4610,4610,4900
Minor,167,399,503,505,56
SqFtTotLiving,990,2950,3389,4100,1570
ZipCode,98188,98040,98040,98040,98146


### So many features!

As data scientists, we should be _very_ cautious about discarding potentially useful data. But, today, we're interested in _only_ the total square footage of each property. What can we do?


In [10]:
bldg_df = bldg_df[['Major', 'Minor', 'SqFtTotLiving', 'ZipCode']]

In [11]:
bldg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511412 entries, 0 to 511411
Data columns (total 4 columns):
Major            511412 non-null int64
Minor            511412 non-null int64
SqFtTotLiving    511412 non-null int64
ZipCode          468372 non-null object
dtypes: int64(3), object(1)
memory usage: 15.6+ MB


In [12]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

### Error!

Why are we seeing an error when we try to join the dataframes?

<table>
    <tr>
        <td style="text-align:left"><pre>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013160 entries, 0 to 2013159
Data columns (total 4 columns):
Major           object
Minor           object
DocumentDate    object
SalePrice       int64
dtypes: int64(1), object(3)
memory usage: 61.4+ MB</pre></td>
        <td style="text-align:left"><pre>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511359 entries, 0 to 511358
Data columns (total 4 columns):
Major            511359 non-null int64
Minor            511359 non-null int64
SqFtTotLiving    511359 non-null int64
ZipCode          468345 non-null object
dtypes: int64(3), object(1)
memory usage: 15.6+ MB
</pre></td>
    </tr>
</table>

Review the error message in light of the above:

* `ValueError: You are trying to merge on object and int64 columns.`

In [29]:
sales_df['Major'] = pd.to_numeric(sales_df['Major'])

### Error!

Note the useful error message above:

`ValueError: Unable to parse string "      " at position 936643`

In this case, we want to treat non-numeric values as missing values. Let's see if there's a way to change how the `pd.to_numeric` function handles errors.

In [ ]:
# The single question mark means "show me the docstring"
pd.to_numeric?

Here's the part that we're looking for:
```
errors : {'ignore', 'raise', 'coerce'}, default 'raise'
    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaN
    - If 'ignore', then invalid parsing will return the input
```

Let's try setting the `errors` parameter to `'coerce'`.

In [30]:
sales_df['Major'] = pd.to_numeric(sales_df['Major'], errors='coerce')

Did it work?

In [31]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014336 entries, 0 to 2014335
Data columns (total 4 columns):
Major           float64
Minor           float64
DocumentDate    object
SalePrice       int64
dtypes: float64(2), int64(1), object(1)
memory usage: 61.5+ MB


It worked! Let's do the same thing with the `Minor` parcel number.

In [32]:
sales_df['Minor'] = pd.to_numeric(sales_df['Minor'], errors='coerce')

In [33]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014336 entries, 0 to 2014335
Data columns (total 4 columns):
Major           float64
Minor           float64
DocumentDate    object
SalePrice       int64
dtypes: float64(2), int64(1), object(1)
memory usage: 61.5+ MB


Now, let's try our join again.

In [34]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

In [35]:
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
0,138860.0,110.0,08/21/2014,245000,1490,98002
1,138860.0,110.0,06/12/1989,109300,1490,98002
2,138860.0,110.0,01/16/2005,14684,1490,98002
3,138860.0,110.0,06/08/2005,0,1490,98002
4,423943.0,50.0,10/11/2014,0,960,98092


In [36]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1437522 entries, 0 to 1437521
Data columns (total 6 columns):
Major            1437522 non-null float64
Minor            1437522 non-null float64
DocumentDate     1437522 non-null object
SalePrice        1437522 non-null int64
SqFtTotLiving    1437522 non-null int64
ZipCode          1322148 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 76.8+ MB


We can see right away that we're missing zip codes for many of the sales transactions. (1321536 non-null entries for ZipCode is fewer than the 1436772 entries in the dataframe.) 

In [37]:
sales_data.loc[sales_data['ZipCode'].isna()].head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
99,858140.0,376.0,05/22/2012,0,900,NaN
100,858140.0,376.0,11/28/2017,0,900,NaN
152,720319.0,520.0,11/20/2013,699950,2840,NaN
153,720319.0,520.0,09/16/2013,0,2840,NaN
163,894677.0,240.0,12/21/2016,818161,2450,NaN


In [38]:
bad_data = sales_data.loc[sales_data['ZipCode'].isna(), :]

bad_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115374 entries, 99 to 1437503
Data columns (total 6 columns):
Major            115374 non-null float64
Minor            115374 non-null float64
DocumentDate     115374 non-null object
SalePrice        115374 non-null int64
SqFtTotLiving    115374 non-null int64
ZipCode          0 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 6.2+ MB


Because we are interested in finding houses in Seattle zip codes, we will need to drop the rows with missing zip codes.

In [39]:
sales_data = sales_data.loc[~sales_data['ZipCode'].isna(), :]

sales_data.head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
0,138860.0,110.0,08/21/2014,245000,1490,98002
1,138860.0,110.0,06/12/1989,109300,1490,98002
2,138860.0,110.0,01/16/2005,14684,1490,98002
3,138860.0,110.0,06/08/2005,0,1490,98002
4,423943.0,50.0,10/11/2014,0,960,98092


In [40]:
sales_data[sales_data['ZipCode'] == 98122]['SalePrice'].mean()

267628.912710115

# Your turn: Data Cleaning with Pandas

### 1. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

Use multiple notebook cells to accomplish this! Press `[esc]` then `B` to create a new cell below the current cell. Press `[return]` to start typing in the new cell.

In [41]:
print('Current total: ' , len(sales_data), '\nNo Sales Value: ',len(sales_data.loc[sales_data['SalePrice'] == 0]), '\nNoSqrtFt: ',len(sales_data.loc[sales_data['SqFtTotLiving'] == 0]))

Current total:  1322148 
No Sales Value:  454732 
NoSqrtFt:  15


In [42]:
sales_data = sales_data.loc[sales_data['SalePrice'] != 0]

In [43]:
sales_data = sales_data.loc[sales_data['SqFtTotLiving'] != 0]

### 2. Investigate and handle non-numeric ZipCode values

Can you find a way to shorten ZIP+4 codes to the first five digits?

What's the right thing to do with missing values?

In [44]:
# Read the error message and decide how to fix it.
# Note: using errors='coerce' is the *wrong* choice in this case.
def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].head()

10168    98042-3001
10169    98042-3001
10170    98042-3001
10171    98042-3001
16528    98199-3014
Name: ZipCode, dtype: object

In [46]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 867407 entries, 0 to 1437520
Data columns (total 6 columns):
Major            867407 non-null float64
Minor            867407 non-null float64
DocumentDate     867407 non-null object
SalePrice        867407 non-null int64
SqFtTotLiving    867407 non-null int64
ZipCode          867407 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 46.3+ MB


In [62]:
sales_data['ZipCode'] = sales_data['ZipCode'].apply(lambda x: str(x)[:5])
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
0,138860.0,110.0,08/21/2014,245000,1490,98002
1,138860.0,110.0,06/12/1989,109300,1490,98002
2,138860.0,110.0,01/16/2005,14684,1490,98002
6,423943.0,50.0,07/15/1999,96000,960,98092
7,423943.0,50.0,01/08/2001,127500,960,98092


### 3. Add a column for PricePerSqFt



In [63]:
sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode,PricePerSqFt
0,138860.0,110.0,08/21/2014,245000,1490,98002,164.429530
1,138860.0,110.0,06/12/1989,109300,1490,98002,73.355705
2,138860.0,110.0,01/16/2005,14684,1490,98002,9.855034
6,423943.0,50.0,07/15/1999,96000,960,98092,100.000000
7,423943.0,50.0,01/08/2001,127500,960,98092,132.812500


### 4. Subset the data to 2019 sales only.

We can assume that the DocumentDate is approximately the sale date.

In [64]:
twenty_nineteen = sales_data.loc[sales_data['DocumentDate'].apply(lambda x: x[6:]) == '2019'].copy()
twenty_nineteen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3626 entries, 605 to 1437220
Data columns (total 7 columns):
Major            3626 non-null float64
Minor            3626 non-null float64
DocumentDate     3626 non-null object
SalePrice        3626 non-null int64
SqFtTotLiving    3626 non-null int64
ZipCode          3626 non-null object
PricePerSqFt     3626 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 226.6+ KB


### 5. Subset the data to zip codes within the City of Seattle.

You'll need to find a list of Seattle zip codes!

In [87]:
twenty_nineteen['ZipCode'].value_counts().head()

98042    134
98023    102
98034    100
98118     94
98058     91
Name: ZipCode, dtype: int64

### 6. What is the mean price per square foot for a house sold in Seattle in 2019?

Don't just type the answer. Type code that generates the answer as output!

In [67]:
print("Price per square foot for home in Seattle in 2019: $", round(twenty_nineteen['PricePerSqFt'].mean(), 2))

Price per square foot for home in Seattle in 2019:  401.47


In [73]:
print("In my Zip Code (98122): $",round(twenty_nineteen[twenty_nineteen['ZipCode'] == '98122']['PricePerSqFt'].mean(), 2))

In my Zip Code (98122): $ 555.88
